In [74]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from pandas import read_csv
from pandas import datetime
from sklearn import *
from tqdm import tqdm_notebook as tqdm
# load dataset
data_attck = pd.read_csv('datasets/training_data/training_data_attacks.csv')

In [75]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel, VarianceThreshold

X = data_attck.values[:,:-1]
y = data_attck.values[:,-1]
selector = VarianceThreshold(threshold=0.01)
X_selected = pd.DataFrame(selector.fit_transform(X))
X_selected.columns = data_attck.columns[selector.get_support(indices=True)]
X_selected.head()

,Unnamed: 0,frame_time_epoch,frame_len,sccp_called_digits,sccp_calling_digits,sccp_calling_ssn,sccp_called_ssn,sccp_message_type,tcap_tid,tcap_dtid,...,gsm_map_ss_forwardedToNumber_4,gsm_map_ss_ussd_DataCodingScheme,gsm_map_ss_ussd_String,gsm_map_ussd_string,gsm_map_ss_SS_Code_4,gsm_map_ss_SS_Code_more,gsm_map_ms_apn_Subscribed,gsm_map_ms_apn_InUse,gsm_map_ms_msisdn_2,gsm_map_ms_msisdn_3
0,0.0,1.518010e+09,212.0,4.910192e+11,4.910190e+11,6.0,7.0,17.0,1.112611e+10,1.112611e+10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.518010e+09,288.0,4.910192e+11,4.910190e+11,6.0,8.0,17.0,5.110001e+09,7.077430e+07,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,1.518010e+09,252.0,4.910192e+11,4.910190e+11,6.0,8.0,17.0,7.077430e+07,7.077430e+07,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,1.518010e+09,352.0,4.910192e+11,4.910190e+11,6.0,7.0,17.0,0.000000e+00,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,1.518010e+09,352.0,4.910192e+11,4.910190e+11,6.0,7.0,17.0,0.000000e+00,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [76]:
X_selected.drop(['Unnamed: 0', 'frame_time_epoch'], axis=1, inplace=True)

In [104]:
useful_features = set()
# finding important features all together is not possible (For some reason),
# intead try to find useful features incrementally
for i in tqdm(range(len(data_attck.columns)-5)):
    try:
        clf = ExtraTreesClassifier(n_estimators=50)
        clf = clf.fit(X_selected.values[:,i:i+6], y)
        feature_importances = pd.DataFrame(clf.feature_importances_,
                                           index = X_selected.columns[i:i+6],
                                            columns=['importance']).sort_values('importance',ascending=False)
        for item in list(feature_importances.index.values[np.where(feature_importances > 0.1)[0]]):
            useful_features.add(item)
    except Exception as e:
        print("failed: ", (i, i+11))
        print(e)
        continue

A Jupyter Widget

failed:  (70, 81)
Input contains NaN, infinity or a value too large for dtype('float32').
failed:  (71, 82)
Input contains NaN, infinity or a value too large for dtype('float32').
failed:  (72, 83)
Input contains NaN, infinity or a value too large for dtype('float32').
failed:  (73, 84)
Input contains NaN, infinity or a value too large for dtype('float32').
failed:  (74, 85)
Input contains NaN, infinity or a value too large for dtype('float32').
failed:  (75, 86)
Input contains NaN, infinity or a value too large for dtype('float32').
failed:  (78, 89)
Input contains NaN, infinity or a value too large for dtype('float32').
failed:  (79, 90)
Input contains NaN, infinity or a value too large for dtype('float32').
failed:  (80, 91)
Input contains NaN, infinity or a value too large for dtype('float32').
failed:  (81, 92)
Input contains NaN, infinity or a value too large for dtype('float32').
failed:  (82, 93)
Input contains NaN, infinity or a value too large for dtype('float32').
failed:  (

In [106]:
# X_selected.loc[:,list(useful_features)]
len(list(useful_features))

60

In [111]:
clf = ExtraTreesClassifier(n_estimators=50)
clf = clf.fit(X_selected.loc[:,list(useful_features)], y)
clf.feature_importances_
# debugging the model fittings
feature_importances = pd.DataFrame(clf.feature_importances_,
                                   index = list(useful_features),
                                    columns=['importance']).sort_values('importance',ascending=False)
print(round(feature_importances, 7))

                                          importance
gsm_map_ms_msc_Number                   8.928379e-01
gsm_map_ms_vlr_Number                   3.805270e-02
tcap_otid                               1.769190e-02
tcap_tid                                1.743020e-02
sccp_called_digits                      7.275100e-03
e164_msisdn                             5.538300e-03
e164_country_code                       5.014300e-03
e212_imsi                               2.711400e-03
gsm_map_ms_imsi                         2.418800e-03
e164_msisdn_2                           2.278100e-03
gsm_old_localValue                      1.350000e-03
frame_len                               1.118000e-03
tcap_application_context_name           1.041400e-03
e164_country_code_2                     1.019900e-03
tcap_localValue                         9.474000e-04
gsm_map_ms_imeisv                       4.532000e-04
gsm_map_ms_supportedLCS_CapabilitySets  4.347000e-04
gsm_map_ms_lcsCapabilitySet3            3.9180

In [112]:
feature_importances

,importance
gsm_map_ms_msc_Number,8.928379e-01
gsm_map_ms_vlr_Number,3.805266e-02
tcap_otid,1.769194e-02
tcap_tid,1.743019e-02
sccp_called_digits,7.275132e-03
e164_msisdn,5.538281e-03
e164_country_code,5.014304e-03
e212_imsi,2.711376e-03
gsm_map_ms_imsi,2.418765e-03
e164_msisdn_2,2.278108e-03
